In [1]:
import os
import cv2
import sys
import random
import time
import argparse
import numpy as np
import matplotlib.pyplot as plt
import copy
import gzip
import pickle
import h5py

import multiprocessing as mp

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from data import load_data, prepare_input, normalize, denormalize
from models import DPINet
from utils import calc_box_init_FluidShake


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--pstep', type=int, default=2)
parser.add_argument('--epoch', type=int, default=0)
parser.add_argument('--iter', type=int, default=0)
parser.add_argument('--env', default='')
parser.add_argument('--time_step', type=int, default=0)
parser.add_argument('--time_step_clip', type=int, default=0)
parser.add_argument('--dt', type=float, default=1./60.)
parser.add_argument('--nf_relation', type=int, default=300)
parser.add_argument('--nf_particle', type=int, default=200)
parser.add_argument('--nf_effect', type=int, default=200)
parser.add_argument('--outf', default='files')
parser.add_argument('--dataf', default='data')
parser.add_argument('--evalf', default='eval')
parser.add_argument('--eval', type=int, default=1)
parser.add_argument('--verbose_data', type=int, default=0)
parser.add_argument('--verbose_model', type=int, default=0)

parser.add_argument('--debug', type=int, default=0)

parser.add_argument('--n_instances', type=int, default=0)
parser.add_argument('--n_stages', type=int, default=0)
parser.add_argument('--n_his', type=int, default=0)

# shape state:
# [x, y, z, x_last, y_last, z_last, quat(4), quat_last(4)]
parser.add_argument('--shape_state_dim', type=int, default=14)

# object attributes:
parser.add_argument('--attr_dim', type=int, default=0)

# object state:
parser.add_argument('--state_dim', type=int, default=0)
parser.add_argument('--position_dim', type=int, default=0)

# relation attr:
parser.add_argument('--relation_dim', type=int, default=0)

_StoreAction(option_strings=['--relation_dim'], dest='relation_dim', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, help=None, metavar=None)

In [3]:
args = parser.parse_args("--env FluidFall --epoch 12 --iter 180000".split())

In [4]:
phases_dict = dict()

env_idx = 4
data_names = ['positions', 'velocities']

# object states:
# [x, y, z, xdot, ydot, zdot]
args.state_dim = 6
args.position_dim = 3

# object attr:
# [fluid]
args.attr_dim = 1

# relation attr:
# [none]
args.relation_dim = 1

args.time_step = 121
args.time_step_clip = 5
args.n_instance = 1
args.n_stages = 1

args.neighbor_radius = 0.08

phases_dict["instance_idx"] = [0, 189]
phases_dict["root_num"] = [[]]
phases_dict["instance"] = ['fluid']
phases_dict["material"] = ['fluid']

args.outf = 'dump_FluidFall/' + args.outf
args.evalf = 'dump_FluidFall/' + args.evalf

args.outf = args.outf + '_' + args.env
args.evalf = args.evalf + '_' + args.env
args.dataf = 'data/' + args.dataf + '_' + args.env

print(args)

Namespace(attr_dim=1, dataf='data/data_FluidFall', debug=0, dt=0.016666666666666666, env='FluidFall', epoch=12, eval=1, evalf='dump_FluidFall/eval_FluidFall', iter=180000, n_his=0, n_instance=1, n_instances=0, n_stages=1, neighbor_radius=0.08, nf_effect=200, nf_particle=200, nf_relation=300, outf='dump_FluidFall/files_FluidFall', position_dim=3, pstep=2, relation_dim=1, shape_state_dim=14, state_dim=6, time_step=121, time_step_clip=5, verbose_data=0, verbose_model=0)


In [5]:
print("Loading stored stat from %s" % args.dataf)
stat_path = os.path.join(args.dataf, 'stat.h5')
stat = load_data(data_names[:2], stat_path)
for i in range(len(stat)):
    stat[i] = stat[i][-args.position_dim:, :]
    # print(data_names[i], stat[i].shape)



Loading stored stat from data/data_FluidFall


In [6]:

use_gpu = torch.cuda.is_available()

model = DPINet(args, stat, phases_dict, residual=True, use_gpu=use_gpu)

if args.epoch == 0 and args.iter == 0:
    model_file = os.path.join(args.outf, 'net_best.pth')
else:
    model_file = os.path.join(args.outf, 'net_epoch_%d_iter_%d.pth' % (args.epoch, args.iter))

print("Loading network from %s" % model_file)
model.load_state_dict(torch.load(model_file))
model.eval()

Loading network from dump_FluidFall/files_FluidFall/net_epoch_12_iter_180000.pth


DPINet(
  (particle_encoder_list): ModuleList(
    (0): ParticleEncoder(
      (model): Sequential(
        (0): Linear(in_features=13, out_features=200, bias=True)
        (1): ReLU()
        (2): Linear(in_features=200, out_features=200, bias=True)
        (3): ReLU()
      )
    )
  )
  (relation_encoder_list): ModuleList(
    (0): RelationEncoder(
      (model): Sequential(
        (0): Linear(in_features=27, out_features=300, bias=True)
        (1): ReLU()
        (2): Linear(in_features=300, out_features=300, bias=True)
        (3): ReLU()
        (4): Linear(in_features=300, out_features=300, bias=True)
        (5): ReLU()
      )
    )
  )
  (relation_propagator_list): ModuleList(
    (0): Propagator(
      (linear): Linear(in_features=700, out_features=200, bias=True)
      (relu): ReLU()
    )
  )
  (particle_propagator_list): ModuleList(
    (0): Propagator(
      (linear): Linear(in_features=400, out_features=200, bias=True)
      (relu): ReLU()
    )
  )
  (rigid_particle_

In [7]:

criterionMSE = nn.MSELoss()

if use_gpu:
    model.cuda()


infos = np.arange(5)



In [8]:
args.verbose_data = 1
args.verbose_model = 1

In [9]:


recs = []
idx = 0

print("Rollout %d / %d" % (idx, len(infos)))
des_dir = os.path.join(args.evalf, 'rollout_%d' % idx)
os.system('mkdir -p ' + des_dir)

# ground truth
for step in range(args.time_step - 1):
    data_path = os.path.join(args.dataf, 'valid', str(infos[idx]), str(step) + '.h5')
    data_nxt_path = os.path.join(args.dataf, 'valid', str(infos[idx]), str(step + 1) + '.h5')

    data = load_data(data_names, data_path)
    data_nxt = load_data(data_names, data_nxt_path)
    velocities_nxt = data_nxt[1]

    if step == 0:
        if args.env == 'BoxBath':
            positions, velocities, clusters = data
            n_shapes = 0
            scene_params = np.zeros(1)
        elif args.env == 'FluidFall':
            positions, velocities = data
            n_shapes = 0
            scene_params = np.zeros(1)
        elif args.env == 'RiceGrip':
            positions, velocities, shape_quats, clusters, scene_params = data
            n_shapes = shape_quats.shape[0]
        elif args.env == 'FluidShake':
            positions, velocities, shape_quats, scene_params = data
            n_shapes = shape_quats.shape[0]
        else:
            raise AssertionError("Unsupported env")

        count_nodes = positions.shape[0]
        n_particles = count_nodes - n_shapes
        print("n_particles", n_particles)
        print("n_shapes", n_shapes)

        p_gt = np.zeros((args.time_step - 1, n_particles + n_shapes, args.position_dim))
        s_gt = np.zeros((args.time_step - 1, n_shapes, args.shape_state_dim))
        v_nxt_gt = np.zeros((args.time_step - 1, n_particles + n_shapes, args.position_dim))

        p_pred = np.zeros((args.time_step - 1, n_particles + n_shapes, args.position_dim))

    p_gt[step] = positions[:, -args.position_dim:]
    v_nxt_gt[step] = velocities_nxt[:, -args.position_dim:]

    # print(step, np.sum(np.abs(v_nxt_gt[step, :args.n_particles])))

    if args.env == 'RiceGrip' or args.env == 'FluidShake':
        s_gt[step, :, :3] = positions[n_particles:, :3]
        s_gt[step, :, 3:6] = p_gt[max(0, step-1), n_particles:, :3]
        s_gt[step, :, 6:10] = data[2]
        s_gt[step, :, 10:] = data[2]

    positions = positions + velocities_nxt * args.dt



Rollout 0 / 5
n_particles 189
n_shapes 0


In [10]:
# model rollout
data_path = os.path.join(args.dataf, 'valid', str(infos[idx]), '0.h5')
data = load_data(data_names, data_path)

for step in range(args.time_step - 1):
    if step % 10 == 0:
        print("Step %d / %d" % (step, args.time_step - 1))

    p_pred[step] = data[0]

    # st_time = time.time()
    attr, state, rels, n_particles, n_shapes, instance_idx = \
            prepare_input(data, stat, args, phases_dict, args.verbose_data)

    Ra, node_r_idx, node_s_idx, pstep = rels[3], rels[4], rels[5], rels[6]

    Rr, Rs = [], []
    for j in range(len(rels[0])):
        Rr_idx, Rs_idx, values = rels[0][j], rels[1][j], rels[2][j]
        Rr.append(torch.sparse.FloatTensor(
            Rr_idx, values, torch.Size([node_r_idx[j].shape[0], Ra[j].size(0)])))
        Rs.append(torch.sparse.FloatTensor(
            Rs_idx, values, torch.Size([node_s_idx[j].shape[0], Ra[j].size(0)])))

    buf = [attr, state, Rr, Rs, Ra]

    with torch.set_grad_enabled(False):
        if use_gpu:
            for d in range(len(buf)):
                if type(buf[d]) == list:
                    for t in range(len(buf[d])):
                        buf[d][t] = Variable(buf[d][t].cuda())
                else:
                    buf[d] = Variable(buf[d].cuda())
        else:
            for d in range(len(buf)):
                if type(buf[d]) == list:
                    for t in range(len(buf[d])):
                        buf[d][t] = Variable(buf[d][t])
                else:
                    buf[d] = Variable(buf[d])

        attr, state, Rr, Rs, Ra = buf
        # print('Time prepare input', time.time() - st_time)

        # st_time = time.time()
        vels = model(
            attr, state, Rr, Rs, Ra, n_particles,
            node_r_idx, node_s_idx, pstep,
            instance_idx, phases_dict, args.verbose_model)
        # print('Time forward', time.time() - st_time)

        # print(vels)

        if args.debug:
            data_nxt_path = os.path.join(args.dataf, 'valid', str(infos[idx]), str(step + 1) + '.h5')
            data_nxt = normalize(load_data(data_names, data_nxt_path), stat)
            label = Variable(torch.FloatTensor(data_nxt[1][:n_particles]).cuda())
            # print(label)
            loss = np.sqrt(criterionMSE(vels, label).item())
            print(loss)

    vels = denormalize([vels.data.cpu().numpy()], [stat[1]])[0]

    if args.env == 'RiceGrip' or args.env == 'FluidShake':
        vels = np.concatenate([vels, v_nxt_gt[step, n_particles:]], 0)
    data[0] = data[0] + vels * args.dt

    if args.env == 'RiceGrip':
        # shifting the history
        # positions, restPositions
        data[1][:, args.position_dim:] = data[1][:, :-args.position_dim]
    data[1][:, :args.position_dim] = vels

    if args.debug:
        data[0] = p_gt[step + 1].copy()
        data[1][:, :args.position_dim] = v_nxt_gt[step]



Step 0 / 120
positions (189, 3)
velocities (189, 3)
n_particles 189
n_shapes 0
Instance_idx: [0, 189]
instance #0 0 189
Attr shape (after add env specific graph components): (189, 1)
Object attr: [189.]
Relations neighbor (5207, 3)
clusters None
Attr shape (after hierarchy building): (189, 1)
Object attr: [189.]
Particle attr: [189.]
Shape attr: [0.]
Roots attr: [0.]
Particle positions stats
(189, 3)
[-1.60331976 -0.23574895 -1.70799009]
[1.05080799 3.06103202 0.97353317]
[-0.35595573  1.71381013 -0.35539217]
[0.66762229 1.17873639 0.66756805]
Velocities stats
(189, 3)
[ 3.34181385e-03 -1.13950720e+00 -4.94587099e-04]
[0.87530558 0.33633816 0.87885342]
0 0 188 0 188
=== Stage 0
Rrp tensor(indices=tensor([[   0,    1,    2,  ..., 5204, 5205, 5206],
                       [   0,    0,    0,  ...,  188,  188,  188]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       device='cuda:0', size=(5207, 189), nnz=5207, layout=torch.sparse_coo)
attr_r torch.Size([189, 7]) state_r torch.

pstep 1
Receiver index range 0 188
Sender index range 0 188
relation effect: torch.Size([5191, 200])
particle effect: torch.Size([189, 200])
pred: torch.Size([189, 3])
positions (189, 3)
velocities (189, 3)
n_particles 189
n_shapes 0
Instance_idx: [0, 189]
instance #0 0 189
Attr shape (after add env specific graph components): (189, 1)
Object attr: [189.]
Relations neighbor (5167, 3)
clusters None
Attr shape (after hierarchy building): (189, 1)
Object attr: [189.]
Particle attr: [189.]
Shape attr: [0.]
Roots attr: [0.]
Particle positions stats
(189, 3)
[-1.63358192 -0.67421202 -1.75116829]
[1.03660405 2.72418881 1.00286206]
[-0.35966442  1.33322437 -0.37152076]
[0.64971223 1.21128055 0.66260443]
Velocities stats
(189, 3)
[-0.07972764 -1.13912347 -0.20464936]
[0.68259905 0.3511069  0.60096877]
0 0 188 0 188
=== Stage 0
Rrp tensor(indices=tensor([[   0,    1,    2,  ..., 5164, 5165, 5166],
                       [   0,    0,    0,  ...,  188,  188,  188]]),
       values=tensor([1., 1., 

pstep 1
Receiver index range 0 188
Sender index range 0 188
relation effect: torch.Size([4903, 200])
particle effect: torch.Size([189, 200])
pred: torch.Size([189, 3])
positions (189, 3)
velocities (189, 3)
n_particles 189
n_shapes 0
Instance_idx: [0, 189]
instance #0 0 189
Attr shape (after add env specific graph components): (189, 1)
Object attr: [189.]
Relations neighbor (4869, 3)
clusters None
Attr shape (after hierarchy building): (189, 1)
Object attr: [189.]
Particle attr: [189.]
Shape attr: [0.]
Roots attr: [0.]
Particle positions stats
(189, 3)
[-1.65556409 -0.79265015 -1.76266885]
[1.04322836 2.35767069 0.96422305]
[-0.37288679  0.9909304  -0.39084274]
[0.68840368 1.16382018 0.69968016]
Velocities stats
(189, 3)
[-0.18931683 -0.36774109 -0.17382263]
[0.7969127  0.71954206 0.82704947]
0 0 188 0 188
=== Stage 0
Rrp tensor(indices=tensor([[   0,    1,    2,  ..., 4866, 4867, 4868],
                       [   0,    0,    0,  ...,  188,  188,  188]]),
       values=tensor([1., 1., 

pstep 1
Receiver index range 0 188
Sender index range 0 188
relation effect: torch.Size([4733, 200])
particle effect: torch.Size([189, 200])
pred: torch.Size([189, 3])
positions (189, 3)
velocities (189, 3)
n_particles 189
n_shapes 0
Instance_idx: [0, 189]
instance #0 0 189
Attr shape (after add env specific graph components): (189, 1)
Object attr: [189.]
Relations neighbor (4721, 3)
clusters None
Attr shape (after hierarchy building): (189, 1)
Object attr: [189.]
Particle attr: [189.]
Shape attr: [0.]
Roots attr: [0.]
Particle positions stats
(189, 3)
[-1.68879711 -0.79250081 -1.83630962]
[1.01374811 2.11706315 1.01201991]
[-0.4089563   0.74841707 -0.41225359]
[0.69759134 1.01396172 0.70852137]
Velocities stats
(189, 3)
[-0.39400861 -0.22857982 -0.22354072]
[0.7057893  0.82550112 0.69247486]
0 0 188 0 188
=== Stage 0
Rrp tensor(indices=tensor([[   0,    1,    2,  ..., 4718, 4719, 4720],
                       [   0,    0,    0,  ...,  188,  188,  188]]),
       values=tensor([1., 1., 

Attr shape (after add env specific graph components): (189, 1)
Object attr: [189.]
Relations neighbor (4527, 3)
clusters None
Attr shape (after hierarchy building): (189, 1)
Object attr: [189.]
Particle attr: [189.]
Shape attr: [0.]
Roots attr: [0.]
Particle positions stats
(189, 3)
[-1.72788628 -0.79372999 -1.80093877]
[0.93014767 2.15348004 0.99283945]
[-0.48233291  0.54811356 -0.45362975]
[0.68304297 0.89870248 0.69487605]
Velocities stats
(189, 3)
[-0.85174217 -0.11475339 -0.26671565]
[1.77739118 0.91692271 0.97056615]
0 0 188 0 188
=== Stage 0
Rrp tensor(indices=tensor([[   0,    1,    2,  ..., 4524, 4525, 4526],
                       [   0,    0,    0,  ...,  188,  188,  188]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       device='cuda:0', size=(4527, 189), nnz=4527, layout=torch.sparse_coo)
attr_r torch.Size([189, 7]) state_r torch.Size([189, 6])
attr_r_rel torch.Size([4527, 7]) attr_s_rel torch.Size([4527, 7]) state_r_rel torch.Size([4527, 6]) state_s_rel torch

...

Rrp tensor(indices=tensor([[   0,    1,    2,  ..., 4920, 4921, 4922],
                       [   0,    0,    0,  ...,  188,  188,  188]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       device='cuda:0', size=(4923, 189), nnz=4923, layout=torch.sparse_coo)
attr_r torch.Size([189, 7]) state_r torch.Size([189, 6])
attr_r_rel torch.Size([4923, 7]) attr_s_rel torch.Size([4923, 7]) state_r_rel torch.Size([4923, 6]) state_s_rel torch.Size([4923, 6]) Ra[s] torch.Size([4923, 1])
relation encode: torch.Size([4923, 300])
pstep 0
Receiver index range 0 188
Sender index range 0 188
relation effect: torch.Size([4923, 200])
particle effect: torch.Size([189, 200])
pstep 1
Receiver index range 0 188
Sender index range 0 188
relation effect: torch.Size([4923, 200])
particle effect: torch.Size([189, 200])
pred: torch.Size([189, 3])
positions (189, 3)
velocities (189, 3)
n_particles 189
n_shapes 0
Instance_idx: [0, 189]
instance #0 0 189
Attr shape (after add env specific graph components):

pstep 1
Receiver index range 0 188
Sender index range 0 188
relation effect: torch.Size([5167, 200])
particle effect: torch.Size([189, 200])
pred: torch.Size([189, 3])
positions (189, 3)
velocities (189, 3)
n_particles 189
n_shapes 0
Instance_idx: [0, 189]
instance #0 0 189
Attr shape (after add env specific graph components): (189, 1)
Object attr: [189.]
Relations neighbor (5177, 3)
clusters None
Attr shape (after hierarchy building): (189, 1)
Object attr: [189.]
Particle attr: [189.]
Shape attr: [0.]
Roots attr: [0.]
Particle positions stats
(189, 3)
[-2.35660189 -0.79780935 -2.43110477]
[1.45875342 0.2509502  1.54768242]
[-0.56336229 -0.51037874 -0.335562  ]
[1.02533004 0.24505753 1.05544193]
Velocities stats
(189, 3)
[-0.03940953  0.50367046 -0.02571388]
[1.15802772 0.58751986 1.10413365]
0 0 188 0 188
=== Stage 0
Rrp tensor(indices=tensor([[   0,    1,    2,  ..., 5174, 5175, 5176],
                       [   0,    0,    0,  ...,  188,  188,  188]]),
       values=tensor([1., 1., 

clusters None
Attr shape (after hierarchy building): (189, 1)
Object attr: [189.]
Particle attr: [189.]
Shape attr: [0.]
Roots attr: [0.]
Particle positions stats
(189, 3)
[-2.51491267 -0.79588649 -2.51345253]
[ 1.55153394 -0.11505089  1.70602284]
[-0.56945184 -0.56434834 -0.34263401]
[1.08155045 0.18873247 1.11846957]
Velocities stats
(189, 3)
[-0.08693843  0.70061788 -0.14459372]
[0.83680062 0.31020366 0.88167926]
0 0 188 0 188
=== Stage 0
Rrp tensor(indices=tensor([[   0,    1,    2,  ..., 5206, 5207, 5208],
                       [   0,    0,    0,  ...,  188,  188,  188]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       device='cuda:0', size=(5209, 189), nnz=5209, layout=torch.sparse_coo)
attr_r torch.Size([189, 7]) state_r torch.Size([189, 6])
attr_r_rel torch.Size([5209, 7]) attr_s_rel torch.Size([5209, 7]) state_r_rel torch.Size([5209, 6]) state_s_rel torch.Size([5209, 6]) Ra[s] torch.Size([5209, 1])
relation encode: torch.Size([5209, 300])
pstep 0
Receiver index 

Attr shape (after add env specific graph components): (189, 1)
Object attr: [189.]
Relations neighbor (5201, 3)
clusters None
Attr shape (after hierarchy building): (189, 1)
Object attr: [189.]
Particle attr: [189.]
Shape attr: [0.]
Roots attr: [0.]
Particle positions stats
(189, 3)
[-2.62507967 -0.79682068 -2.60192596]
[ 1.61506144 -0.2299676   1.74232822]
[-0.57598557 -0.5931335  -0.37168512]
[1.13094486 0.16421897 1.16452428]
Velocities stats
(189, 3)
[-0.00373756  0.7977085  -0.30088674]
[0.59519618 0.17082805 0.55039521]
0 0 188 0 188
=== Stage 0
Rrp tensor(indices=tensor([[   0,    1,    2,  ..., 5198, 5199, 5200],
                       [   0,    0,    0,  ...,  188,  188,  188]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       device='cuda:0', size=(5201, 189), nnz=5201, layout=torch.sparse_coo)
attr_r torch.Size([189, 7]) state_r torch.Size([189, 6])
attr_r_rel torch.Size([5201, 7]) attr_s_rel torch.Size([5201, 7]) state_r_rel torch.Size([5201, 6]) state_s_rel to

attr_r torch.Size([189, 7]) state_r torch.Size([189, 6])
attr_r_rel torch.Size([5167, 7]) attr_s_rel torch.Size([5167, 7]) state_r_rel torch.Size([5167, 6]) state_s_rel torch.Size([5167, 6]) Ra[s] torch.Size([5167, 1])
relation encode: torch.Size([5167, 300])
pstep 0
Receiver index range 0 188
Sender index range 0 188
relation effect: torch.Size([5167, 200])
particle effect: torch.Size([189, 200])
pstep 1
Receiver index range 0 188
Sender index range 0 188
relation effect: torch.Size([5167, 200])
particle effect: torch.Size([189, 200])
pred: torch.Size([189, 3])
positions (189, 3)
velocities (189, 3)
n_particles 189
n_shapes 0
Instance_idx: [0, 189]
instance #0 0 189
Attr shape (after add env specific graph components): (189, 1)
Object attr: [189.]
Relations neighbor (5179, 3)
clusters None
Attr shape (after hierarchy building): (189, 1)
Object attr: [189.]
Particle attr: [189.]
Shape attr: [0.]
Roots attr: [0.]
Particle positions stats
(189, 3)
[-2.67055836 -0.79894042 -2.71085678]
[ 

In [ ]:
import pyflex
pyflex.init()


##### render for the ground truth
pyflex.set_scene(env_idx, scene_params, 0)

if args.env == 'RiceGrip':
    halfEdge = np.array([0.15, 0.8, 0.15])
    center = np.array([0., 0., 0.])
    quat = np.array([1., 0., 0., 0.])
    pyflex.add_box(halfEdge, center, quat)
    pyflex.add_box(halfEdge, center, quat)
elif args.env == 'FluidShake':
    x, y, z, dim_x, dim_y, dim_z, box_dis_x, box_dis_z = scene_params
    boxes = calc_box_init_FluidShake(box_dis_x, box_dis_z, height, border)

    x_box = x + (dim_x-1)/2.*0.055

    for box_idx in range(len(boxes) - 1):
        halfEdge = boxes[box_idx][0]
        center = boxes[box_idx][1]
        quat = boxes[box_idx][2]
        pyflex.add_box(halfEdge, center, quat)


for step in range(args.time_step - 1):
    if args.env == 'RiceGrip':
        pyflex.set_shape_states(s_gt[step])
    elif args.env == 'FluidShake':
        pyflex.set_shape_states(s_gt[step, :-1])

    mass = np.zeros((n_particles, 1))
    if args.env == 'RiceGrip':
        p = np.concatenate([p_gt[step, :n_particles, -3:], mass], 1)
    else:
        p = np.concatenate([p_gt[step, :n_particles], mass], 1)

    pyflex.set_positions(p)
    pyflex.render(capture=1, path=os.path.join(des_dir, 'gt_%d.tga' % step))



In [ ]:
##### render for the predictions
pyflex.set_scene(env_idx, scene_params, 0)

if args.env == 'RiceGrip':
    pyflex.add_box(halfEdge, center, quat)
    pyflex.add_box(halfEdge, center, quat)
elif args.env == 'FluidShake':
    for box_idx in range(len(boxes) - 1):
        halfEdge = boxes[box_idx][0]
        center = boxes[box_idx][1]
        quat = boxes[box_idx][2]
        pyflex.add_box(halfEdge, center, quat)

for step in range(args.time_step - 1):
    if args.env == 'RiceGrip':
        pyflex.set_shape_states(s_gt[step])
    elif args.env == 'FluidShake':
        pyflex.set_shape_states(s_gt[step, :-1])

    mass = np.zeros((n_particles, 1))
    if args.env == 'RiceGrip':
        p = np.concatenate([p_pred[step, :n_particles, -3:], mass], 1)
    else:
        p = np.concatenate([p_pred[step, :n_particles], mass], 1)

    pyflex.set_positions(p)
    pyflex.render(capture=1, path=os.path.join(des_dir, 'pred_%d.tga' % step))
pyflex.clean()

In [11]:
idx = 0
step = 0

data_path = os.path.join(args.dataf, 'valid', str(infos[idx]), str(step) + '.h5')
data_nxt_path = os.path.join(args.dataf, 'valid', str(infos[idx]), str(step + 1) + '.h5')

data = load_data(data_names, data_path)
data_nxt = load_data(data_names, data_nxt_path)

In [12]:
attr, state, rels, n_particles, n_shapes, instance_idx = \
        prepare_input(data, stat, args, phases_dict, args.verbose_data)

positions (189, 3)
velocities (189, 3)
n_particles 189
n_shapes 0
Instance_idx: [0, 189]
instance #0 0 189
Attr shape (after add env specific graph components): (189, 1)
Object attr: [189.]
Relations neighbor (5207, 3)
clusters None
Attr shape (after hierarchy building): (189, 1)
Object attr: [189.]
Particle attr: [189.]
Shape attr: [0.]
Roots attr: [0.]
Particle positions stats
(189, 3)
[-1.60331976 -0.23574895 -1.70799009]
[1.05080799 3.06103202 0.97353317]
[-0.35595573  1.71381013 -0.35539217]
[0.66762229 1.17873639 0.66756805]
Velocities stats
(189, 3)
[ 3.34181385e-03 -1.13950720e+00 -4.94587099e-04]
[0.87530558 0.33633816 0.87885342]
0 0 188 0 188


In [13]:
print ('attr', attr.shape)
print ('state', state.shape)
print ('n_particles', n_particles)
print ('instance_idx', instance_idx)

attr torch.Size([189, 1])
state torch.Size([189, 6])
n_particles 189
instance_idx [0, 189]


In [14]:
Ra, node_r_idx, node_s_idx, pstep = rels[3], rels[4], rels[5], rels[6]
print ('Ra',Ra)
print ('node_r_idx', node_r_idx)
print ('node_s_idx', node_s_idx)
print ('pstep', pstep)
print ('len(Ra)',Ra[0].shape)

Ra [tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])]
node_r_idx [array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158,

In [15]:
Rr, Rs = [], []
for j in range(len(rels[0])):
    Rr_idx, Rs_idx, values = rels[0][j], rels[1][j], rels[2][j]
    Rr.append(torch.sparse.FloatTensor(
        Rr_idx, values, torch.Size([node_r_idx[j].shape[0], Ra[j].size(0)])))
    Rs.append(torch.sparse.FloatTensor(
        Rs_idx, values, torch.Size([node_s_idx[j].shape[0], Ra[j].size(0)])))

buf = [attr, state, Rr, Rs, Ra]

In [16]:
n_stage = len(Rr)
n_stage

1

In [17]:
attr_r = attr[node_r_idx[0]]
state_r = state[node_r_idx[0]]
state_r.shape

torch.Size([189, 6])

In [18]:
offset = Variable(torch.zeros((attr.size(0), state.size(1))))
offset

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [19]:
Rr

[tensor(indices=tensor([[   0,    0,    0,  ...,  188,  188,  188],
                        [   0,    1,    2,  ..., 5204, 5205, 5206]]),
        values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
        size=(189, 5207), nnz=5207, layout=torch.sparse_coo)]

In [20]:
Rs

[tensor(indices=tensor([[   0,    1,    2,  ...,  186,  187,  188],
                        [   0,    1,    2,  ..., 5204, 5205, 5206]]),
        values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
        size=(189, 5207), nnz=5207, layout=torch.sparse_coo)]

In [ ]:
189*189

In [ ]:
rels

In [ ]:
n_particles

In [ ]:

for idx in range(len(infos)):

    print("Rollout %d / %d" % (idx, len(infos)))
    des_dir = os.path.join(args.evalf, 'rollout_%d' % idx)
    os.system('mkdir -p ' + des_dir)

    # ground truth
    for step in range(args.time_step - 1):
        data_path = os.path.join(args.dataf, 'valid', str(infos[idx]), str(step) + '.h5')
        data_nxt_path = os.path.join(args.dataf, 'valid', str(infos[idx]), str(step + 1) + '.h5')

        data = load_data(data_names, data_path)
        data_nxt = load_data(data_names, data_nxt_path)
        velocities_nxt = data_nxt[1]

        if step == 0:
            if args.env == 'BoxBath':
                positions, velocities, clusters = data
                n_shapes = 0
                scene_params = np.zeros(1)
            elif args.env == 'FluidFall':
                positions, velocities = data
                n_shapes = 0
                scene_params = np.zeros(1)
            elif args.env == 'RiceGrip':
                positions, velocities, shape_quats, clusters, scene_params = data
                n_shapes = shape_quats.shape[0]
            elif args.env == 'FluidShake':
                positions, velocities, shape_quats, scene_params = data
                n_shapes = shape_quats.shape[0]
            else:
                raise AssertionError("Unsupported env")

            count_nodes = positions.shape[0]
            n_particles = count_nodes - n_shapes
            print("n_particles", n_particles)
            print("n_shapes", n_shapes)

            p_gt = np.zeros((args.time_step - 1, n_particles + n_shapes, args.position_dim))
            s_gt = np.zeros((args.time_step - 1, n_shapes, args.shape_state_dim))
            v_nxt_gt = np.zeros((args.time_step - 1, n_particles + n_shapes, args.position_dim))

            p_pred = np.zeros((args.time_step - 1, n_particles + n_shapes, args.position_dim))

        p_gt[step] = positions[:, -args.position_dim:]
        v_nxt_gt[step] = velocities_nxt[:, -args.position_dim:]

        # print(step, np.sum(np.abs(v_nxt_gt[step, :args.n_particles])))

        if args.env == 'RiceGrip' or args.env == 'FluidShake':
            s_gt[step, :, :3] = positions[n_particles:, :3]
            s_gt[step, :, 3:6] = p_gt[max(0, step-1), n_particles:, :3]
            s_gt[step, :, 6:10] = data[2]
            s_gt[step, :, 10:] = data[2]

        positions = positions + velocities_nxt * args.dt

    # model rollout
    data_path = os.path.join(args.dataf, 'valid', str(infos[idx]), '0.h5')
    data = load_data(data_names, data_path)

    for step in range(args.time_step - 1):
        if step % 10 == 0:
            print("Step %d / %d" % (step, args.time_step - 1))

        p_pred[step] = data[0]

        if args.env == 'RiceGrip' and step == 0:
            data[0] = p_gt[step + 1].copy()
            data[1] = np.concatenate([v_nxt_gt[step]] * (args.n_his + 1), 1)
            continue

        # st_time = time.time()
        attr, state, rels, n_particles, n_shapes, instance_idx = \
                prepare_input(data, stat, args, phases_dict, args.verbose_data)

        Ra, node_r_idx, node_s_idx, pstep = rels[3], rels[4], rels[5], rels[6]

        Rr, Rs = [], []
        for j in range(len(rels[0])):
            Rr_idx, Rs_idx, values = rels[0][j], rels[1][j], rels[2][j]
            Rr.append(torch.sparse.FloatTensor(
                Rr_idx, values, torch.Size([node_r_idx[j].shape[0], Ra[j].size(0)])))
            Rs.append(torch.sparse.FloatTensor(
                Rs_idx, values, torch.Size([node_s_idx[j].shape[0], Ra[j].size(0)])))

        buf = [attr, state, Rr, Rs, Ra]

        with torch.set_grad_enabled(False):
            if use_gpu:
                for d in range(len(buf)):
                    if type(buf[d]) == list:
                        for t in range(len(buf[d])):
                            buf[d][t] = Variable(buf[d][t].cuda())
                    else:
                        buf[d] = Variable(buf[d].cuda())
            else:
                for d in range(len(buf)):
                    if type(buf[d]) == list:
                        for t in range(len(buf[d])):
                            buf[d][t] = Variable(buf[d][t])
                    else:
                        buf[d] = Variable(buf[d])

            attr, state, Rr, Rs, Ra = buf
            # print('Time prepare input', time.time() - st_time)

            # st_time = time.time()
            vels = model(
                attr, state, Rr, Rs, Ra, n_particles,
                node_r_idx, node_s_idx, pstep,
                instance_idx, phases_dict, args.verbose_model)
            # print('Time forward', time.time() - st_time)

            # print(vels)

            if args.debug:
                data_nxt_path = os.path.join(args.dataf, 'valid', str(infos[idx]), str(step + 1) + '.h5')
                data_nxt = normalize(load_data(data_names, data_nxt_path), stat)
                label = Variable(torch.FloatTensor(data_nxt[1][:n_particles]).cuda())
                # print(label)
                loss = np.sqrt(criterionMSE(vels, label).item())
                print(loss)

        vels = denormalize([vels.data.cpu().numpy()], [stat[1]])[0]

        if args.env == 'RiceGrip' or args.env == 'FluidShake':
            vels = np.concatenate([vels, v_nxt_gt[step, n_particles:]], 0)
        data[0] = data[0] + vels * args.dt

        if args.env == 'RiceGrip':
            # shifting the history
            # positions, restPositions
            data[1][:, args.position_dim:] = data[1][:, :-args.position_dim]
        data[1][:, :args.position_dim] = vels

        if args.debug:
            data[0] = p_gt[step + 1].copy()
            data[1][:, :args.position_dim] = v_nxt_gt[step]

    ##### render for the ground truth
    pyflex.set_scene(env_idx, scene_params, 0)

    if args.env == 'RiceGrip':
        halfEdge = np.array([0.15, 0.8, 0.15])
        center = np.array([0., 0., 0.])
        quat = np.array([1., 0., 0., 0.])
        pyflex.add_box(halfEdge, center, quat)
        pyflex.add_box(halfEdge, center, quat)
    elif args.env == 'FluidShake':
        x, y, z, dim_x, dim_y, dim_z, box_dis_x, box_dis_z = scene_params
        boxes = calc_box_init_FluidShake(box_dis_x, box_dis_z, height, border)

        x_box = x + (dim_x-1)/2.*0.055

        for box_idx in range(len(boxes) - 1):
            halfEdge = boxes[box_idx][0]
            center = boxes[box_idx][1]
            quat = boxes[box_idx][2]
            pyflex.add_box(halfEdge, center, quat)


    for step in range(args.time_step - 1):
        if args.env == 'RiceGrip':
            pyflex.set_shape_states(s_gt[step])
        elif args.env == 'FluidShake':
            pyflex.set_shape_states(s_gt[step, :-1])

        mass = np.zeros((n_particles, 1))
        if args.env == 'RiceGrip':
            p = np.concatenate([p_gt[step, :n_particles, -3:], mass], 1)
        else:
            p = np.concatenate([p_gt[step, :n_particles], mass], 1)

        pyflex.set_positions(p)
        pyflex.render(capture=1, path=os.path.join(des_dir, 'gt_%d.tga' % step))

    ##### render for the predictions
    pyflex.set_scene(env_idx, scene_params, 0)

    if args.env == 'RiceGrip':
        pyflex.add_box(halfEdge, center, quat)
        pyflex.add_box(halfEdge, center, quat)
    elif args.env == 'FluidShake':
        for box_idx in range(len(boxes) - 1):
            halfEdge = boxes[box_idx][0]
            center = boxes[box_idx][1]
            quat = boxes[box_idx][2]
            pyflex.add_box(halfEdge, center, quat)

    for step in range(args.time_step - 1):
        if args.env == 'RiceGrip':
            pyflex.set_shape_states(s_gt[step])
        elif args.env == 'FluidShake':
            pyflex.set_shape_states(s_gt[step, :-1])

        mass = np.zeros((n_particles, 1))
        if args.env == 'RiceGrip':
            p = np.concatenate([p_pred[step, :n_particles, -3:], mass], 1)
        else:
            p = np.concatenate([p_pred[step, :n_particles], mass], 1)

        pyflex.set_positions(p)
        pyflex.render(capture=1, path=os.path.join(des_dir, 'pred_%d.tga' % step))

pyflex.clean()



In [ ]:
idx = 0
step = 0

data_path = os.path.join(args.dataf, 'valid', str(infos[idx]), str(step) + '.h5')
data_nxt_path = os.path.join(args.dataf, 'valid', str(infos[idx]), str(step + 1) + '.h5')

data = load_data(data_names, data_path)
data_nxt = load_data(data_names, data_nxt_path)

In [ ]:
positions, velocities = data

In [ ]:
positions.shape

In [ ]:
velocities.shape